<a href="https://colab.research.google.com/github/SarahBarbosa/AluraDataChallenge_2/blob/main/S02_Feature_Engineering_e_Modelo_de_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://imgur.com/QnT31vZ.png)

---

# Sobre o desafio

Na semana 02, vamos focar na construção e otimização de modelos de machine learning (ML) para lidar com a Taxa de Churn. Agora que terminamos a limpeza e análise inicial dos dados na semana 01, é hora de usar algoritmos de ML para transformar nossos insights em ações eficazes.

## Configurando o ambiente

In [14]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Github/AluraDataChallenge_2')

## Carregando os dados tratados na Semana 01

In [15]:
import pandas as pd

DIRETORIO_BASE = '/content/drive/MyDrive/Github/AluraDataChallenge_2/'
DIRETORIO_DADOS = DIRETORIO_BASE + 'Dados/'

dados_novexus = pd.read_csv(DIRETORIO_DADOS + 'dados_novexus_tratado.csv')

**Observação:** Antes de avançar com nossas análises e modelagem, vamos modificar o nome das colunas para simplificar nosso trabalho (poderíamos ter feito isso na semana 01, mas ainda dá tempo):

In [16]:
import funcoes_auxiliares as func_aux

dados = dados_novexus.rename(columns = func_aux.renomeia_coluna).copy()

In [17]:
dados.columns

Index(['Churn', 'Gender', 'Seniorcitizen', 'Partner', 'Dependents', 'Tenure',
       'Multiplelines', 'Onlinesecurity', 'Onlinebackup', 'Deviceprotection',
       'Techsupport', 'Streamingtv', 'Streamingmovies', 'Paperlessbilling',
       'Charges_monthly', 'Charges_total', 'Internetservice_dsl',
       'Internetservice_fiber_optic', 'Contract_month-to-month',
       'Contract_one_year', 'Contract_two_year',
       'Paymentmethod_bank_transfer_automatic',
       'Paymentmethod_credit_card_automatic', 'Paymentmethod_electronic_check',
       'Paymentmethod_mailed_check', 'Nophoneservice', 'Nointernetservice'],
      dtype='object')

<div style="background-color: #171821; color: white; padding: 10px; text-align: center; font-size: 20px;">
    <h1>Feature Engineering</h1>
</div>

Após as análises detalhadas feitas na semana 01, ficou claro que podemos criar novas variáveis e segmentação dos dados para tornar nossas features mais úteis para a construção dos modelos ML. No entanto, por boas práticas, criamos um transformador (veja `funcoes_feat_eng.py`) com um hiperparâmetro que bloqueia a adição ou não de novas features para casos que não temos 100% de certeza se essa nova feature irá ajudar ou não no algoritmo de ML.

E quais seriam essas novas features?


## Experimentando combinações de features

### Criando segmentos anuais para Tenure e segmentos para Charges_monthly e Charges_total

Sabemos que a variável `Tenure` representa o período de contrato em meses de cada cliente. Para uma melhor análise, decidimos categorizar essa feature em intervalos anuais. Vale mencionar que o intervalo mínimo é de 1 mês, enquanto o máximo é de 72 meses (6 anos).

> Categorias: `['0-1 Year', '1-2 Year', '2-3 Year', '3-4 Year', '4-5 Year', '5-6 Year']`

Além disso, podemos fazer uma segmentação nos gastos mensais (`Charges_Monthly`) e nas gastos totais (`Charges_Total`) em 4 categorias. Essa abordagem nos permite identificar grupos de clientes com diferentes níveis de despesas, o que é mais informativo do que considerar apenas valores contínuos.

> Categorias: `['Low', 'Moderate', 'High', 'Very High']`

*Observação*: deixamos em inglês pois as colunas não foram traduzidas.



### Número total de serviços de internet recebidos pelo cliente

Calculamos o número total de serviços recebidos por cada cliente com base em várias condições relacionadas aos serviços de internet. Isso nos dá uma visão geral do alcance dos serviços utilizados pelos clientes.

### Calculando o custo por serviço

Por fim, para determinar o custo por serviço, dividimos o valor mensal dos gastos pelo número total de serviços mais um (adicionando "+1" para evitar divisões por zero quando não há nenhum serviço contratado).

## Encoding adicional e Feature Scaling

Com base nos insights anteriores, agora é necessário converter as novas features em valores binários, usando a função `pd.get_dummies()` que anteriormente evitamos. Além disso, precisamos ajustar os dados que estão em uma escala consideravelmente maior do que 0 e 1 para que todos estejam na mesma escala. Esse processo de ajuste nos permitirá padronizar as features, removendo a média e dimensionando para uma variação unitária.

## "Pipeline" de transformação

In [12]:
from sklearn.preprocessing import StandardScaler
from funcoes_feat_eng import ProcessadorDadosCustomizado

# Adicionando novas features
processador = ProcessadorDadosCustomizado()
dados_extras = processador.transform(dados)

# Encoding One-Hot
dados_extras = pd.get_dummies(dados_extras)

# Feature Scaling
colunas_numericas = [col for col in dados_extras.columns if dados_extras[col].nunique() > 2]
scaler = StandardScaler()
dados_extras[colunas_numericas] = scaler.fit_transform(dados_extras[colunas_numericas])

In [21]:
dados_extras[colunas_numericas].head()

,Tenure,Charges_monthly,Charges_total,Num_internet_service,charge_per_internet_service
0,-0.954310,0.027862,-0.745055,0.520738,-0.645034
1,-0.954310,-0.161583,-0.767522,-0.561776,0.316483
2,-1.158162,0.303720,-0.882969,-0.561776,0.813208
3,-0.791228,1.104706,-0.460553,1.061995,-0.417960
4,-1.198932,0.636080,-0.888905,-0.020519,0.175745


<div style="background-color: #171821; color: white; padding: 10px; text-align: center; font-size: 20px;">
    <h1>Lidando com o desbalanceamento de dados da target</h1>
</div>

Durante a primeira semana do nosso projeto, uma observação veio à tona: nossos dados da target 'Churn' apresentam um desbalanceamento, conforme evidenciado abaixo:

In [22]:
round(dados_extras['Churn'].value_counts(normalize = True) * 100, 2)

0    73.46
1    26.54
Name: Churn, dtype: float64

Este desbalanceamento pode trazer desafios na construção de modelos ML. Na verdade, temos duas abordagens distintas a serem consideradas para lidar com essa situação:

## Abordagem 1: Ignorar o desbalanceamento

> A primeira abordagem consiste em ignorar completamente o desbalanceamento e construir nosso modelo sem fazer ajustes específicos para corrigir. Se desenvolvermos um modelo sem considerar essa desproporcionalidade nos dados, o modelo será vítima do Paradoxo da Acurácia, em que os parâmetros do algoritmo não diferenciarão a classe minoritária das demais categorias, acreditando que estão agregando resultado devido à aparente alta acurácia. No entanto, existem diversos algoritmos de classificação, como Gradient Boosting, por exemplo, que demonstram um desempenho melhor lidando com dados desbalanceados do que modelos como o KNN e SVM.


In [23]:
X_ignore = dados_extras.drop('Churn', axis = 1)   # Conjunto de features
y_ignore = dados_extras['Churn']                  # Conjunto da target Churn

## Abordagem 2: Oversamplig

> A segunda abordagem é utilizar a técnica [SMOTE](https://arxiv.org/pdf/1106.1813.pdf) (Synthetic Minority Over-sampling Technique). Essa técnica envolve a criação de informações sintéticas com base nas observações já existentes na classe minoritária. Esses dados "sintéticos" são gerados de forma a serem semelhantes aos dados reais, embora não sejam idênticos. Essa abordagem visa equilibrar a representação das classes, tornando o modelo mais capaz de identificar corretamente os casos da classe minoritária.

In [24]:
from imblearn.over_sampling import SMOTE

X_oversample, y_oversample = SMOTE(random_state= 42).fit_resample(X_ignore, y_ignore)

In [26]:
round(y_oversample.value_counts(normalize = True) * 100, 2)

0    50.0
1    50.0
Name: Churn, dtype: float64

A partir deste ponto, nosso trabalho se ramificará nessas duas abordagens distintas. Ao final, identificaremos qual delas é a mais eficaz, juntamente com a escolha do modelo de classificação adequado. Também discutiremos as medidas que a empresa Novexus pode adotar para reduzir a taxa de Churn.